In [2]:
from typing import Iterator, List, Union, Tuple
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Model
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
import os



In [2]:
dataset = ('C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2')

In [3]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'out.csv'))
# # Split data into training and validation sets
df_train, df_val = train_test_split(df, test_size=.2)

In [32]:
datagen=ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True)


In [56]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="File", y_col="angle", 
                                            class_mode="raw", target_size=(480,640), batch_size=32, shuffle=True)

Found 4491 validated image filenames.


In [57]:
val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, x_col="File", y_col="angle", 
                                            class_mode="raw", target_size=(480,640), batch_size=32, shuffle=True)

Found 1123 validated image filenames.


In [26]:
input_shape = (480, 640, 3)

In [60]:
from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import pooling
from keras.layers import Dropout


In [63]:
# define PilotNet model, with batch normalization included.
def get_model(input_shape):
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(axis=1),
        pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
#         BatchNormalization(axis=1),
        pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
#         BatchNormalization(axis=1),
        pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
#         BatchNormalization(axis=1),
        pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu',name='target_layer'),
#         BatchNormalization(axis=1),
        pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Flatten(),
        Dense(100, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
#         BatchNormalization(),
        Dropout(.5),
        Dense(50, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
#         BatchNormalization(),
        Dropout(.25),
        Dense(10, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
#         BatchNormalization(),
        Dense(1)
    ])
    
    return model

model = get_model(input_shape)
#sgd = SGD(learning_rate=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-04), loss="mse",  metrics=[keras.metrics.MeanSquaredError()]) 
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 238, 318, 24)      1824      
_________________________________________________________________
batch_normalization_5 (Batch (None, 238, 318, 24)      952       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 119, 159, 24)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 58, 78, 36)        21636     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 39, 36)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 13, 18, 48)        43248     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 9, 48)         

In [64]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [65]:
model_name = 'regression_udacity2'

In [66]:
callback1 = EarlyStopping(monitor="val_loss", patience = 3) 
#         min_delta=1,  # model should improve by at least 1%
#          # amount of epochs  with improvements worse than 1% until the model stops
#         verbose=2,
#         mode="min",
#         restore_best_weights=True) # restore the best model with the lowest validation error)

In [67]:
callback2 = ModelCheckpoint( "./models/" + model_name + ".h5",
        monitor="val_loss",
        verbose=0,
        save_best_only=True,  # save the best model
        mode="min",
        save_freq="epoch") # save every epoch
    

In [14]:
import tf_explain
from tf_explain.core.grad_cam import GradCAM


In [15]:
callback3 = tf_explain.callbacks.GradCAMCallback(val_generator, layer_name='target_layer', class_index=0)




In [71]:
model.load_weights('udacity_regression2_v2.h5')

In [72]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=4, callbacks=[callback1])

Epoch 1/4
140/140 [==============================] - 1078s 8s/step - loss: 0.1295 - mean_squared_error: 0.0390 - val_loss: 0.1248 - val_mean_squared_error: 0.0398
Epoch 2/4
140/140 [==============================] - 1047s 7s/step - loss: 0.1175 - mean_squared_error: 0.0374 - val_loss: 0.1135 - val_mean_squared_error: 0.0382
Epoch 3/4
140/140 [==============================] - 1045s 7s/step - loss: 0.1048 - mean_squared_error: 0.0337 - val_loss: 0.0985 - val_mean_squared_error: 0.0315
Epoch 4/4
140/140 [==============================] - 1031s 7s/step - loss: 0.0921 - mean_squared_error: 0.0287 - val_loss: 0.0865 - val_mean_squared_error: 0.0266


In [73]:
model.save('udacity_regression2_v3.h5')

In [14]:
from keras.applications import inception_v3 as inc_net
import numpy as np

In [13]:
from keras.preprocessing import image


In [18]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img((img_path), target_size=(480,640))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out) 

In [30]:
images1 = transform_img_fn([os.path.join(r'C:/Users/Ketan/Thesis/self-driving-car/challenges/challenge-2/','1479425721831580656.jpg')])


In [31]:
model1.predict(images1)

array([[-2.1896603]], dtype=float32)

In [59]:
scores = model.evaluate_generator(val_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


evaluate_generator mean_squared_error: 1.42%


In [80]:
test_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, x_col="File", y_col="angle", 
                                            class_mode="raw", target_size=(480,640), batch_size=1, shuffle=True)

Found 1123 validated image filenames.


In [81]:
filenames = test_generator.filenames
nb_samples = len(filenames)


In [83]:
predict = model.predict_generator(test_generator,steps = nb_samples)


In [87]:
predict

array([[ 0.06131682],
       [ 0.07231429],
       [-0.10049643],
       ...,
       [ 0.16951463],
       [-0.00924093],
       [ 0.24610852]], dtype=float32)

In [6]:
from PIL import Image
import numpy as np

In [23]:
def open_images(inference_folder: str) -> np.ndarray:
    """Loads images from a folder and prepare them for inferencing.

    Parameters
    ----------
    inference_folder : str
        Location of images for inferencing.

    Returns
    -------
    np.ndarray
        List of images as numpy arrays transformed to fit the efficient_net model input specs.
    """
    images = []
    for img in os.listdir(inference_folder):
        img_location = os.path.join(inference_folder, img)  # create full path to image

        with Image.open(img_location) as img:  # open image with pillow

            img = np.array(img)
            img = img[:, :, :3]
            img = np.expand_dims(img, axis=0)  # add 0 dimension to fit input shape of efficient_net
            img = inc_net.preprocess_input(img)

        images.append(img)
    images_array = np.vstack(images)  # combine images efficiently to a numpy array
    return images_array

In [24]:
images = open_images("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset")


In [99]:
predictions = model.predict_generator(images)

images_names = os.listdir("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset")
for image_name, prediction in zip(images_names, predictions):
    print(image_name, prediction)

1479425719681268780.jpg [0.99608827]
1479425719731264417.jpg [0.88264745]
1479425719781262693.jpg [0.7628715]
1479425719831276356.jpg [0.71570003]
1479425719881419974.jpg [0.46559352]
1479425719931414543.jpg [0.183625]
1479425719981321507.jpg [0.34010532]
1479425720031300599.jpg [0.28711617]
1479425720081346893.jpg [0.51028806]
1479425720131269683.jpg [0.5584253]
1479425720181373910.jpg [0.7031558]
1479425720231334733.jpg [0.85264647]
1479425720281331591.jpg [1.0384414]
1479425720331302374.jpg [0.99889576]
1479425720381363046.jpg [1.0292076]
1479425720431335470.jpg [0.9913622]
1479425720481412110.jpg [0.9542479]
1479425720531376518.jpg [0.9778542]
1479425720581393603.jpg [1.0917541]
1479425720631398027.jpg [1.1867032]
1479425720681517212.jpg [1.0979418]
1479425720731420306.jpg [1.0765804]
1479425720781425602.jpg [1.1168288]
1479425720831471166.jpg [0.9714207]
1479425720881517725.jpg [1.1084675]
1479425720931436208.jpg [1.2131143]
1479425720981458366.jpg [1.2013582]
1479425721031504367.

In [10]:
from keras.initializers import glorot_uniform


In [27]:
# with keras.utils.custom_object_scope({'GlorotUniform': glorot_uniform()}):
model1 = keras.models.load_model('tl_regression_udacity_v1.h5', compile=False)

In [25]:
predictions1 = model1.predict_generator(images)

images_names = os.listdir("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset")
for image_name, prediction in zip(images_names, predictions1):
    print(image_name, predictions1)

1479425719681268780.jpg [[-2.9998062]
 [-2.9467986]
 [-2.7615364]
 [-2.7122169]
 [-2.5220637]
 [-2.5943952]
 [-2.4407299]
 [-2.4350517]
 [-2.3533406]
 [-2.168322 ]
 [-1.9964895]
 [-1.9248927]
 [-1.8540998]
 [-1.8204676]
 [-2.0903869]
 [-2.2754335]
 [-2.4574442]
 [-2.4135587]
 [-2.1645963]
 [-1.9479976]
 [-1.7416089]
 [-1.714652 ]
 [-1.4838266]
 [-1.895048 ]
 [-1.927601 ]
 [-1.7706724]
 [-1.6582344]
 [-1.8991219]
 [-1.8153596]
 [-1.6163443]
 [-2.0196428]
 [-2.1415842]
 [-2.242585 ]
 [-2.3724556]
 [-2.377391 ]
 [-2.3197978]
 [-2.3446722]
 [-2.1966772]
 [-2.546969 ]
 [-2.4286554]
 [-2.2052805]
 [-2.4023674]
 [-2.3768215]
 [-2.189649 ]
 [-2.3940697]]
1479425719731264417.jpg [[-2.9998062]
 [-2.9467986]
 [-2.7615364]
 [-2.7122169]
 [-2.5220637]
 [-2.5943952]
 [-2.4407299]
 [-2.4350517]
 [-2.3533406]
 [-2.168322 ]
 [-1.9964895]
 [-1.9248927]
 [-1.8540998]
 [-1.8204676]
 [-2.0903869]
 [-2.2754335]
 [-2.4574442]
 [-2.4135587]
 [-2.1645963]
 [-1.9479976]
 [-1.7416089]
 [-1.714652 ]
 [-1.4838266]

In [ ]:
model1.predict()